<a href="https://colab.research.google.com/github/analuisagc/preprocess4pm/blob/main/Pre%CC%81_Processamento_PM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pré-processamento de Arquivo Excel com Python e Pandas

Neste notebook, vamos demonstrar como realizar o pré-processamento de um arquivo Excel utilizando a biblioteca `pandas` do Python. O pré-processamento incluirá a leitura do arquivo, tratamento de valores ausentes, renomeação de colunas e conversão de tipos de dados.

* <strong>Autora: </strong>Ana Luisa Seleme</p>
* <p><strong>Data de criação: </strong>27/05/2024</p>
* **Bibliotecas necessárias**:

1.   Pandas
2.   Google.colab
3.   ipywidgets
4.   IPython

In [ ]:
import pandas as pd #importa a biblioteca Pandas
from google.colab import files #importa a biblioteca do Google Colab
import ipywidgets as widgets # biblioteca para criação do widget de seleção de múltiplas colunas
from IPython.display import display # biblioteca para que usuário interaja com a aplicação

#abre um "select box" para selecionar o arquivo em Excel a ser processado
uploaded = files.upload()
file_name = next(iter(uploaded))

#faz o upload do arquivo em Excel para o Google Colab
df = pd.read_excel(file_name)

Saving obitos-2018-2020 (1).xlsx to obitos-2018-2020 (1).xlsx


In [ ]:
# Remover a coluna "Unnamed: 0" se existir
if 'Unnamed: 0' in df.columns:
    df.drop(columns=['Unnamed: 0'], inplace=True)

# Obter a primeira coluna do DataFrame
primeira_coluna = df.columns[0]

# Função para criar widgets de seleção de colunas
def create_column_selectors(df, index, activity_default, caseid_default):
    columns = df.columns.tolist()

    caseid_selector = widgets.Dropdown(
        options=[''] + columns,
        value=caseid_default,
        description=f'CaseID DF{index}:',
        disabled=False
    )

    activity_selector = widgets.Dropdown(
        options=["sintomas", "admissão", "óbito"],
        value=activity_default,
        description=f'Activity DF{index}:',
        disabled=False
    )

    date_selector = widgets.Dropdown(
        options=[''] + columns,
        description=f'Date DF{index}:',
        disabled=False
    )

    time_selector = widgets.Dropdown(
        options=[''] + columns,
        description=f'Time DF{index} (optional):',
        disabled=False
    )

    resource_selector = widgets.Dropdown(
        options=[''] + columns,
        description=f'Resource DF{index}:',
        disabled=False
    )

    return [caseid_selector, activity_selector, date_selector, time_selector, resource_selector]

# Definir os valores padrão para cada `activity_selector`
activities = ["sintomas", "admissão", "óbito"]
selectors = [create_column_selectors(df, i, activities[i-1], primeira_coluna) for i in range(1, 4)]

# Layout para exibir os widgets de seleção lado a lado
selectors_layout = widgets.HBox([widgets.VBox(selectors[i]) for i in range(3)])

# Função auxiliar para padronizar datas e horas
def parse_datetime(datetime_str):
    for fmt in ("%Y-%m-%d %H:%M", "%d/%m/%Y %H:%M", "%Y-%m-%d", "%d/%m/%Y"):
        try:
            return datetime.strptime(datetime_str, fmt).strftime('%Y-%m-%d %H:%M')
        except ValueError:
            continue
    return datetime_str  # Retorna o texto original se não conseguir converter

# Função para criar um DataFrame com as colunas mapeadas
def create_mapped_dataframe():
    mapped_dfs = []
    for i, selectors_set in enumerate(selectors, 1):
        caseid_selector, activity_selector, date_selector, time_selector, resource_selector = selectors_set

        # Verificar se a coluna de hora foi informada e substituir valores inválidos
        if time_selector.value == '' or time_selector.value not in df.columns:
            time_series = pd.Series(['00:00'] * len(df))
        else:
            time_series = df[time_selector.value].fillna('00:00')
            time_series = time_series.replace('', '00:00')

        # Concatenar data e hora e converter para datetime
        date_time_series = df[date_selector.value].astype(str) + ' ' + time_series.astype(str)
        timestamp_series = date_time_series.apply(parse_datetime)

        df_mapped = pd.DataFrame({
            'CaseID': df[caseid_selector.value],
            'Activity': [activity_selector.value] * len(df),
            'TimeStamp': timestamp_series,
            'Resource': df[resource_selector.value]
        })

        mapped_dfs.append(df_mapped)

    # Concatenar todos os DataFrames e ordenar por `CaseID`
    final_df = pd.concat(mapped_dfs, axis=0, ignore_index=True)
    final_df.sort_values(by='CaseID', inplace=True)
    return final_df

# Função para exportar o DataFrame para Excel
def export_to_excel(df):
    df.to_excel('Data_Log.xlsx', index=False)
    files.download('Data_Log.xlsx')
    print("DataLog exportado para Excel com sucesso!")

# Botão para criar o DataFrame mapeado
create_button = widgets.Button(description="Criar DataLog")

# Função a ser executada quando o botão for clicado
def on_create_button_clicked(b):
    df_mapped = create_mapped_dataframe()
    export_button = widgets.Button(description="Exportar para Excel")
    print("DataLog criado com as colunas mapeadas e ordenado por CaseID:")
    display(df_mapped.head())
    display(export_button)
    export_button.on_click(lambda x: export_to_excel(df_mapped))

create_button.on_click(on_create_button_clicked)

# Exibir os seletores e o botão para criar DataFrame
display(selectors_layout)
display(create_button)

Button(description='Criar DataLog', style=ButtonStyle())